In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

Task1 
1. Load CIFAR1-10 dataset

In [4]:
# 1. Load CIFAR-10 Dataset
transform = transforms.ToTensor()
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, activation_fn=nn.LeakyReLU()):
        super(SimpleCNN, self).__init__()
        # The first layer has 3 input channels and 32 output channels. The point here is
        # that the CIFR images are 32 * 32, and RGB channels after the to tensor. So, we take
        self.conv_layer = nn.Conv2d(3, 32, kernel_size=5) 
        # Fully connected layer - the hidden layers are 28 * 28 hence and pooling reduces it by 2
        # Hence, 14 * 14 and mult that with 32 feature/conv layers.
        self.nn_layer = nn.Linear(32 * 14 * 14, 10)
        self.activation = activation_fn  # Activation function
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
    
    def forward(self, x):
        x = self.pool(self.activation(self.conv_layer(x))) 
        x = torch.flatten(x, 1)  # Flatten the feature maps, as the next layer is 1d layer
        x = self.nn_layer(x)  # Output layer
        return x

# ANN-1 excercise
def train_model(model, optimizer, error_func, device, epochs=10, log_dir=""):
    writer = SummaryWriter(log_dir)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in trainloader:
            
            images, labels = images.to(device), labels.to(device) 
            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = error_func(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total  # Compute accuracy
        writer.add_scalar('Loss/train', running_loss / len(trainloader), epoch)  # Log loss
        writer.add_scalar('Accuracy/train', train_acc, epoch)  # Log accuracy
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}, Accuracy: {train_acc}%')
    writer.close()

# Train model with LeakyReLU and SGD optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The device I have is {device}")

In [ ]:


model1 = SimpleCNN(nn.LeakyReLU())
optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model1, optimizer1, err_func,device, 10, log_dir='logs/leakyrelu_sgd')

# Train model with LeakyReLU and Adam optimizer
model2 = SimpleCNN(nn.LeakyReLU())
optimizer2 = optim.Adam(model2.parameters())
train_model(model2, optimizer2, err_func,device, 10, log_dir='logs/leakyrelu_adam')

# Train model with Tanh and Adam optimizer
model3 = SimpleCNN(nn.Tanh())
optimizer3 = optim.Adam(model3.parameters())
train_model(model3, optimizer3, err_func,device, 10, log_dir='logs/tanh_adam')



The device I have is cuda
Epoch 1, Loss: 2.2921436086030265, Accuracy: 11.284%
Epoch 2, Loss: 2.271450419560113, Accuracy: 14.464%
Epoch 3, Loss: 2.25336359010633, Accuracy: 17.906%
Epoch 4, Loss: 2.2353316332068283, Accuracy: 20.622%
Epoch 5, Loss: 2.2172058659136447, Accuracy: 22.97%
Epoch 6, Loss: 2.198722933869228, Accuracy: 24.628%
Epoch 7, Loss: 2.180347995380002, Accuracy: 25.308%
Epoch 8, Loss: 2.1620438855017543, Accuracy: 26.248%
Epoch 9, Loss: 2.1438465722076727, Accuracy: 26.966%
Epoch 10, Loss: 2.1261131671993323, Accuracy: 27.726%
Epoch 1, Loss: 1.5117358376302987, Accuracy: 47.088%
Epoch 2, Loss: 1.2426141701695863, Accuracy: 56.812%
Epoch 3, Loss: 1.1364305840276392, Accuracy: 60.57%
Epoch 4, Loss: 1.0608168920440137, Accuracy: 63.564%
Epoch 5, Loss: 1.012735387019794, Accuracy: 65.064%
Epoch 6, Loss: 0.9734480774311154, Accuracy: 66.47%
Epoch 7, Loss: 0.9400296862930289, Accuracy: 67.88%
Epoch 8, Loss: 0.9114635889335057, Accuracy: 68.922%
Epoch 9, Loss: 0.891923534367

In [13]:
# 2. Fine-tuning the AlexNet model for CIFAR-10
transform_alex = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 to match AlexNet input size
    transforms.ToTensor()])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_alex)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_alex)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

alexnet = models.alexnet(pretrained=False)  # We will initialize AlexNet without pre-trained weights
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10) 
optimizer_alexnet = optim.Adam(alexnet.parameters())
err_func = nn.CrossEntropyLoss()
# device = torch.device("cpu")

train_model(alexnet, optimizer_alexnet, err_func,device, 10, log_dir='logs/alexnet_finetuning')

alexnet_pretrained = models.alexnet(pretrained=True)


alexnet_pretrained.classifier[6] = nn.Linear(in_features=4096, out_features=10)
optimizer_pretrained = optim.Adam(alexnet_pretrained.classifier[6].parameters())

train_model(alexnet_pretrained, optimizer_pretrained, err_func,device, 10, log_dir='logs/alexnet_feature_extraction')

d:\LTU\AdvancedNN\ann_lab_0\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\LTU\AdvancedNN\ann_lab_0\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1, Loss: 1.8377353067288313, Accuracy: 31.568%
Epoch 2, Loss: 1.4956363430413444, Accuracy: 45.296%
Epoch 3, Loss: 1.3241985467693689, Accuracy: 52.264%
Epoch 4, Loss: 1.1965012149432737, Accuracy: 57.324%
Epoch 5, Loss: 1.1025679047455264, Accuracy: 60.99%
Epoch 6, Loss: 1.0127191164761857, Accuracy: 64.388%
Epoch 7, Loss: 0.9604864958149698, Accuracy: 66.442%
Epoch 8, Loss: 0.8999955250174189, Accuracy: 68.474%
Epoch 9, Loss: 0.8698861631743439, Accuracy: 69.58%
Epoch 10, Loss: 0.833780252720084, Accuracy: 70.962%


d:\LTU\AdvancedNN\ann_lab_0\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\srini/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100.0%


Epoch 1, Loss: 1.196297695950779, Accuracy: 57.842%


KeyboardInterrupt: 

##### Cancelled the pre-trained alexnet, after 1 iteration, as it was taking a long time.

In [15]:
#Using the same CNN for MINST and SVHN datsets.


transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match your model's expected input size
    transforms.Grayscale(num_output_channels=3),  # Convert 1 channel -> 3 channels
    transforms.ToTensor()
])

mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

trainloader = DataLoader(mnist_trainset, batch_size=64, shuffle=True)
testloader = DataLoader(mnist_testset, batch_size=64, shuffle=False)


model_minst = SimpleCNN(nn.Tanh())
optimizer_minst = optim.Adam(model_minst.parameters())

# optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model_minst, optimizer_minst, err_func,device, 5, log_dir='logs/minst_tanh_adam')

Epoch 1, Loss: 0.219141412510483, Accuracy: 93.54333333333334%
Epoch 2, Loss: 0.08491279584452954, Accuracy: 97.545%
Epoch 3, Loss: 0.06569448769227096, Accuracy: 98.02833333333334%
Epoch 4, Loss: 0.05691579594317802, Accuracy: 98.30166666666666%
Epoch 5, Loss: 0.0471664274515741, Accuracy: 98.52166666666666%


In [1]:
transform_svhn = transforms.Compose(transforms.ToTensor())

svhn_trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)
svhn_testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform_svhn)

svhn_trainloader = DataLoader(svhn_trainset, batch_size=64, shuffle=True)
svhn_testloader = DataLoader(svhn_testset, batch_size=64, shuffle=False)

model_svhn = SimpleCNN(nn.Tanh())
optimizer_svhn = optim.Adam(model_svhn.parameters())

# optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model_svhn, optimizer_svhn, err_func,device, 5, log_dir='logs/svhn_tanh_adam')

NameError: name 'transforms' is not defined